# Setting Up the API Connections
We need to first install the OpenAi package from pip (if not already installed)

```pip
pip install openai
```

The OpenAI API uses API keys for authentication. Visit your API Keys page on openai.com to retrieve the API key you'll use in your requests.

Remember that your API key is a secret! Do not share it with others or expose it in any client-side code (browsers, apps). Production requests must be routed through your own backend server where your API key can be securely loaded from an environment variable or key management service.

Best practice is to store the key in an environment variable using this method inside a .custom_bashrc file (THAT IS NOT STORED IN ANY PUBLIC REPO!!!):

```bash
export OPENAI_API_KEY=<your_key_here>
```

All API requests should include your API key in an Authorization HTTP header as follows:

```
Authorization: Bearer OPENAI_API_KEY
```

Here is an example of a basic request to the API. This request queries the gpt-3.5-turbo model to complete the text starting with a prompt of "Say this is a test".

In [1]:
import requests
import json
import os
import textwrap
import openai
from PIL import Image
from IPython.display import display
import io

assert os.environ.get('OPENAI_API_KEY') is not None, 'ERROR, your environment variable OPENAI_API_KEY is not set properly'

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.Model.list()

<OpenAIObject list at 0x126675bc0> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampl

## Now lets try the API using multiple responses and a high temperature

In [3]:
prompt = "Monster made up of nothing but teeth and limbs devouring the World"
# how many distinct responses to generate
response_num = 2
# A higher value of temperature results in a more randomized distribution of probabilities, 
# which can lead to more surprising and diverse text. 
# Conversely, a lower value of temperature results in a more conservative and predictable distribution of probabilities, 
# which can lead to more coherent and consistent text.
# Typical values range from 0.0 to 2.0, where 0.0 results in the same text each time
temperature = 1.2
max_tokens = 40
responses = []

headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + os.environ.get('OPENAI_API_KEY')
}

data = {
    'model': 'gpt-3.5-turbo',
    'messages': [{'role': 'user', 'content': 'Please rewrite this text in one sentence to be more descriptive with a maximum of {} words: {}'.format(max_tokens, prompt)}],
    'temperature': temperature, # TODO - what is temperature?
    'n': response_num, # produce three responses
    'max_tokens': max_tokens * response_num # limit number of tokens to be used
}

# When you pass a Python object to json.dumps(), 
# the function converts it to a JSON string representation of the object. 

# Change the api endpoint to "text-generation" as we dont want to have a conversation with GPT
responses.append(requests.post('https://api.openai.com/v1/chat/completions', headers=headers, data=json.dumps(data)))

print(responses[-1].json()["choices"])

[{'message': {'role': 'assistant', 'content': 'A terrifying creature with a body consisting solely of gnashing teeth and powerful limbs relentlessly consuming everything in its path, bringing about the end of the world.'}, 'finish_reason': 'stop', 'index': 0}, {'message': {'role': 'assistant', 'content': 'A terrifying and formidable monster with gnashing teeth and countless limbs causing widespread destruction and engulfing the entire planet in its ravenous consumption.'}, 'finish_reason': 'stop', 'index': 1}]


In [4]:
for i in range(len(responses[-1].json()["choices"])):
    print(responses[-1].json()["choices"][i]['message']['content'])

A terrifying creature with a body consisting solely of gnashing teeth and powerful limbs relentlessly consuming everything in its path, bringing about the end of the world.
A terrifying and formidable monster with gnashing teeth and countless limbs causing widespread destruction and engulfing the entire planet in its ravenous consumption.


## Great, now lets give it a try with the same prompt, but the "edit" endpoint instead

In [13]:
temperature = 0.1
prompt = "Fantasy painting of a great battle of domestic cats with the black cats triumphent over the yellow tabbys"
# PLEASE NOTE THE NEW MODEL: 
# You can use the text-davinci-edit-001 
# or code-davinci-edit-001 model with this endpoint.
data = {
    'model': 'text-davinci-edit-001',
    'input': prompt,
    'instruction':'Please rewrite this text in one sentence to be more descriptive with a maximum of {} words'.format(max_tokens),
    'temperature': temperature, # TODO - what is temperature?
    'n': response_num # produce three responses
}

responses.append(requests.post('https://api.openai.com/v1/edits', headers=headers, data=json.dumps(data)))

In [11]:
# Now lets print all the responses we generated so far =)
print(responses[-1])
json_string = json.dumps(responses[-1], indent=4)
for response in responses:
    if len(response.json()["choices"]) > 1:
        for r in range(len(response.json()["choices"])):
            print(response.json()["choices"][i]['message']['content'])
    # else:
    #    print(response.json()["choices"][0]['message']['content'])

<Response [200]>


TypeError: Object of type Response is not JSON serializable

In [12]:
print(responses[-1].text)

{"object":"edit","created":1681544833,"choices":[{"text":"I need help.\nApparently cats in Vietnam wish to use the LAN connection to China, and as such are revolting as slow as possible against Vietnamese bloodthirsty governments.\nAlthough not all these cats are pure female (filly, estrones and sex prisoners), fighting cats worship each cute puppy ever born, congratulating their Lord the Truant Thatchame if they spot before the doomed revolution stands.\n","index":0},{"text":"Fantasy painting of a great battle of domestic cats with the black cats triumphent over the yellow tabbys\n\ntestIFC171_MAUST_ZUREKBD3_IBC12 has challenges. Please place files in GH1 README Author description Purpose separate in single file and folders for each task separately. \n\nTherefore push 10 commits show slide to your article from Wikipedia homeworkGitHub Presentation und HH repo make it private allow zzu87 we give outlines your slides copy make 0 master AH renmae presentation private add Abdoul IBC push 

## Now Lets Use the API to generate some Images

In [ ]:
# Image generation
openai.Image.create(
  prompt=prompt,
  n=2,
  size="1024x1024"
)

In [ ]:
# Open the images
def displayImgsSideBySide(path1, path2):
    image1 = Image.open(path1)
    image2 = Image.open(path2)

    # Resize the images if needed
    image1 = image1.resize((300, 300))
    image2 = image2.resize((300, 300))

    # Create a new blank image with a width equal to the combined width of the two images
    combined_image = Image.new('RGB', (image1.width + image2.width, image1.height))

    # Paste the images side by side into the combined image
    combined_image.paste(image1, (0, 0))
    combined_image.paste(image2, (image1.width, 0))

    # Display the combined image in the IPython notebook
    display(combined_image)
    
displayImgsSideBySide(
    "gpt_cat_battles_01.png", 
    "gpt_cat_battles_02.png")



## Now lets try to feed our own images in for editing
Note the **mask** parameter which is:

- An additional image whose fully transparent areas (e.g. where alpha is zero) indicate where image should be edited. Must be a valid PNG file, less than 4MB, and have the same dimensions as

Also note the response_format parameter which is:

- The format in which the generated images are returned. Must be one of url or b64_json.

In [ ]:
# Open and process the main image
image1 = Image.open("gpt_cat_battles_01.png").convert("RGBA")
image1_bytes = io.BytesIO()
image1.save(image1_bytes, "PNG")
image1_bytes.seek(0)

# Open the mask image and convert the red channel to alpha
image2 = Image.open("gpt_cat_battles_02.png")
r, g, b = image2.split()
image2_alpha = Image.merge("RGBA", (r, g, b, r))
image2_bytes = io.BytesIO()
image2_alpha.save(image2_bytes, "PNG")
image2_bytes.seek(0)

# Call the OpenAI API
response = openai.Image.create_edit(
    image=image1_bytes,
    mask=image2_bytes,
    prompt=prompt,
    n=1,
    size="1024x1024"
)

In [ ]:
print(response)

# Audio Transcription

In [ ]:
mp3_path = "phs_abstract.mp3"
openai.api_key = os.getenv("OPENAI_API_KEY")
audio_file = open(mp3_path, "rb")
transcript = openai.Audio.transcribe(
    "whisper-1", 
    audio_file, 
    prompt="transcribe this audio recording in english to english", 
    language='en')
print(transcript)

# Language Translation

In [ ]:
mp3_path = "phs_abstract.mp3"
audio_file = open(mp3_path, "rb")
translation = openai.Audio.translate(
    "whisper-1", 
    audio_file, 
    prompt="Translate this from english to spanish",
    language='en')

print(translation)


## Mood to Color

In [ ]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt,
  temperature=0,
  max_tokens=64,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[";"]
)
print(response)

## Horror Story Creator

In [ ]:
prompt = "Monster made up of nothing but teeth and limbs"

response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt,
  temperature=0.8,
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.5,
  presence_penalty=0.0
)

print(response['choices'][0]['text'])

In [ ]:
# Image generation

openai.Image.create(
  prompt=response["choices"][0]["text"],
  n=2,
  size="1024x1024"
)

Example output:

In [ ]:
print(response['choices'][0]['text'])
displayImgsSideBySide("gpt_moster_01.png", "gpt_moster_02.png")